# 第6章　バックアップとリストア

## 6-1　pg_dump、pg_dumpall、pg_restore

バックアップには、2通りの方法がある。

平文形式：pg_dump、pg_dumpall（リストアは、psqlコマンド）<br>
平文形式以外：pg_dump（リストアは、pg_restore）

### 6-1-1　pg_dump、pg_dumpallによるバックアップ

オプションは以下の4つとなる。

-Fp：出力形式（Format）として**平文（plain）**を指定する、-Fの指定がない場合のデフォルト<br>
-Fc：出力形式（Format）として**カスタム形式（custom）**を指定する<br>
-Ft：出力形式（Format）として**tar形式（tar）**を指定する<br>
-f：バックアップ先のファイル名を指定する、省略した場合は標準出力となる

In [ ]:
# データベース（examdb）のバックアップを標準出力を出力
pg_dump examdb

In [ ]:
# カスタム形式で、バックアップをファイル（examdb.dump）に出力
pg_dump -Fc examdb -f examdb.dump

In [ ]:
# -fを省略し、標準出力のリダイレクトを行う
pg_dump -Fc examdb < examdb.dump

データベース全体に対するバックアップは、pg_dumpallを使用すれば良い。

In [ ]:
pg_dumpall -f db.sql

### 6-1-2　psqlコマンドを使った、平文形式のリストア

In [ ]:
# バックアップファイル（db.sql）をリストア（復元・展開）する
psql -f db.sql

### 6-1-3　pg_restoreを使った平文形式意外のリストア

-d：リストア先のデータベース名を指定する

In [ ]:
# カスタム形式のバックアップファイル（exam.dump）からリストアする
pg_restore -d examdb examdb.dump

In [ ]:
# -dオプションでデータベース名を指定しなかった場合は、平文形式のSQLコマンドが標準出力に出力される
pg_restore examdb.dump

### 6-1-4　バックアップファイル＆リストアの組み合わせの例

#### 平文でのバックアップ＆リストアの例

In [ ]:
# examdbをファイル名（examdb.sql）でバックアップ
pg_dump -f examdb.sql examdb

dropdb examdb
createdb examdb

# データベース（examdb）にファイル（examdb.sql）を復元する
psql -f examdb.sql examdb

#### カスタム形式でのバックアップ＆リストアの例

In [ ]:
# カスタム形式でexamdbをファイル名（examdb.dump）でバックアップ
pg_dump -Fc examdb -f examdb.dump

dropdb examdb
createdb examdb

# examdb.dumpをデータベース（examdb）に展開する
pg_restore -d examdb examdb.dump

**psql**は**-f**でファイル名を指定し、**pg_restore**は**-d**でデータベース名を指定する。

## 6-2　ディレクトコピーによるバックアップ＆リストア

サーバーを停止した状態でデータベースクラスタをバックアップ・リストアする方法もある。<br>
これは、物理的なディレクトリ全体をコピーするバックアップ方法である。

バックアップにも、下記のような種類がある。

・物理バックアップ：データベース上のテーブル構造やデータとは無関係に取得するバックアップ<br>
・論理バックアップ：テーブル構造やデータを意識したバックアップ

・オンラインバックアップ（ホットバックアップ）：データベースの稼働中に実行するバックアップ<br>
・オフラインバックアップ（コールドバックアップ）：データベースを停止させて実行するバックアップ

例

In [ ]:
# PostgreSQLを停止させる
pg_ctl stop

# ディレクトリのコピーにtarコマンドを使用し、backup.tarファイルにバックアップを取得する
cd $PGDATA/..
tar cvf backup.tar data

例

In [ ]:
pg_ctl status
---
no sever running

cd /home/postgres
ls
---
backup.tar

# tarコマンドで、物理的にバックアップしたファイル（backup.tar）からリストアする
tar xvf backup.tar
ls
---
backup.tar data

pg_ctl start
---
server starting

## 6-3　PITR（point In Time Recovery）

### 6-3-1　PITRの概要

PITRでは、下記が取得される。

・ベースバックアップ：ある時点でのデータベース全体のバックアップ<br>
・WALファイル：最新のログ<br>
・WALアーカイブ：古くなったログ<br>

これらをまとめると、下記のようになる。

・バックアップ＝ベースバックアップ＋WALファイル（最新のログ）＋WALアーカイブ（古くなったログ）

WALファイルがいっぱいになると、古いものから順にWALアーカイブとして別の場所に保管される。

・**ロールフォワードリカバリ**：ベースバックアップをもとに、WALアーカイブ＋WALファイルを用いて最新の状態にリカバリする方法

バックアップ＆リカバリの流れは、下記となる。

①前準備／設定：WALアーカイブの準備、postgres.confの設定<br>
②ベースバックアップ：pg_start_backup()とpg_stop_backup()<br>
③リカバリ：ベースバックアップのリストア→未アーカイブのWALファイルをコピー→recovery.confの設定、PostgreSQLの起動

### 6-3-2　PITRによるバックアップの前準備／設定

ベースバックアップを行うための準備をする。

postgres.confファイルに設定する主なパラメータは、以下となる。

・wal_level：WALに書き込まれる情報の度合い、デフォルト値はreplica、推奨はreplica、logical（minimalは使用しない）<br>
・archive_mode：WALアーカイブを有効にするかの指定、デフォルトはoff、PITR使用時はonにする<br>
・archive_command：WALファイルをWALアーカイブとしてコピーするためのシェルコマンドを指定する

例

In [ ]:
# WALアーカイブを有効にする
achive_mode = on

# シェルコマンドを使用して、古いWALファイルをWALアーカイブにコピーする
'cp %p /mnt/archivedir/%f'

In [ ]:
# WALアーカイブを確認する
ls /mnt/archivedir

# WALファイルを見て、同じ内容がWALアーカイブにコピーされていることを確認する
ls $PGDATA/pg_wal

PostgreSQLの運用中に、WALファイルのデータが満杯になると、WALアーカイブへのコピーが自動で行われる。

### 6-3-3　PITRによるベースバックアップ

実際にベースバックアップの取得をしていく。

尚、PITRで使用するベースバックアップを取得する際は、スーパーユーザで実行する必要がある。

In [ ]:
cd $PGDATA/..

# ラベル名（label）でバックアップの開始
psql -c "select pg_start_backup('label', true);"

# tarコマンドで、tar.gzにベースバックアップを保存
tar czvf /mnt/backup.tar.gz.data

# バックアップの終了
psql -c "seledt pg_stop_backup()"

### 6-3-4　PITRによるリカバリの手順

取得したベースバックを用いて、実際にリカバリ（復旧）をしていく。<br>
これは、PostgreSQLが停止した状態で行わなければいけない。

リストアとリカバリの違いは、以下の通りとなる。

リストア：バックアップしたデータを用いて、そのまま復元すること<br>
リカバリ：バックアップ後に何らかの処理（WALファイルを用いて更新 など）を行い、データを正常化・最新化すること

#### ①ベースバックアップのリストア

In [ ]:
cd $PGDATA/..

# PostgreSQLの停止を確認
pg_ctl_status
---
no server running

# データベースクラスタを移動
mv $PGDATA /mnt

# 移動されていることを確認
ls data

# ベースバックアップを展開
tar xzvf /mnt/backup.tar.gz

#### ②未アーカイブのWALファイルをコピー

WALアーカイブ（古いWAL）に加え、WALファイル（最新のWAL）も復元させるため、未アーカイブのWALファイルをコピーする。<br>
ベースバックアップには、前回バックアップした古いWALファイルがあるので、これを削除しておく。

・古いWALファイルを削除しない場合：現在のWALアーカイブ分のみが加わるので、その時点までの復元となる<br>
・古いWALファイルを削除した場合：現在のWALアーカイブに加え、最新のWALファイルまで加わるので、直前の状態までの復元となる

※教科書では、前者についてWALアーカイブの「最新」の状態と書いてあるが<br>
　これはあくまでWALアーカイブ（古いWAL）の中で最も新しい状態であることに注意する

In [ ]:
# WALアーカイブの一覧
ls /mnt/arvhivedir

rm -rf $PGDATA/pg_wal

# 未アーカイブのWALファイルのコピー
cp -r /mnt/data/pg_wal $PGDATA

#### ③recovery.confファイルを設定して、PostgreSQLを起動

recovery.confファイルは、リカバリに関する設定ファイルである。<br>
これを用いて、バックアップ後の処理（WALファイルやWALアーカイブの更新）を行う。

In [ ]:
cat $PGDATA/recovery.conf

# PostgreSQLの起動
pg_ctl start

これでリカバリが開始される。<br>
終了すると、recovery.confファイルはrecovery.doneという名前のファイルにリネームされる。

In [ ]:
ls $PGDATA/recovery.*
---
/home/postgres/data/recovery.done

## 6-4　COPY文と \copyコマンド

テーブルのデータをファイルへコピー、もしくはファイルデータをテーブルへローディングする方法が2つある。

・COPY：サーバー側で動作する、**スーパーユーザ権限**もしくは**デフォルトロール権限**が必要、ファイル名を**絶対パス**で指定<br>
・\copy：クライアント側で動作する、スーパーユーザ権限は必要なし、ファイル名は絶対パスでも相対パスでも構わない

\copyはネットワーク通信が発生するため、大量のデータであれば、COPYの方が望ましい。

### 6-4-1　COPY文

COPY文には、以下のオプションがある。

・DELIMITER '区切り文字'：各行の列を区切る文字を指定する<br>
・FORMAT csv：CSV形式で入出力する場合に指定する<br>
・HEADER：ヘッダ行の指定、onにすると各列の名前が出力されて先頭行の入力は無効となる（CSV形式を指定した場合のみ有効）

In [ ]:
# テーブル（tab1）のデータを、サーバー側のファイル（/tmp/tab1.txt）へコピーする
COPY tab1 TO '/tmp/tab1.txt';
---
COPY 60

In [ ]:
# サーバー側のファイル（tmp/tab1.txt）からテーブル（tab1）にデータをローディングする
COPY tab1 FROM '/tmp/tab1.txt'
---
COPY 60

### 6-4-2　\copyコマンド　

In [ ]:
select * from tab1;

# テーブル（tab1）のデータを、クライアント側のファイルへコピーする
\copy tab1 to stdout csv header

# クライアント側のファイル（tab1.txt）に、テーブル（tab1）のデータをコピーする
\copy tab1 to tab1.txt csv header
\q
cat tab1.txt

In [ ]:
# オプションにdelimiterを指定して、区切り文字「|」（ストローク）を指定する
\copy tab1 to stdout delimiter as '|'

In [ ]:
# CSV形式でデータが格納されているクライアント側のファイル（tab1.txt）を、テーブル（tab1）にローディングする
\copy tab1 from tab1.txt csv